In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import matrixprofile as mp
from matplotlib.patches import Rectangle
from fbprophet import Prophet
from prophet.plot import plot_yearly
from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import TimeSeriesSplit
from numpy import sqrt
import datetime
from datetime import timedelta
import plotly.graph_objs as go
from sklearn.metrics import r2_score
import re

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/may_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,5,1,2.3,V,...,3.82,V,2.58,V,1.8,V,1.17,V,1.47,V


# Obtenemos datos de temperatura de la estación Chamberí
### Punto de muestreo de Chamartín 28079111, estación 111 
### Punto de muestreo de Centro 28079110, estación 110
### Punto de muestreo de Chamberí 28079109, estación 109
### Debido a que no se encuentra la estación 111 se procede a utilizar los datos de la estación Chamberí

In [3]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]

In [4]:
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]

In [5]:
DataTemp.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
1147,28,79,109,83,28079109_83_98,2020,5,1,13.4,V,...,20.3,V,18.8,V,17.7,V,17.0,V,16.4,V


# Obtenemos datos de Precipitación de la estación Hortaleza 
## Las estaciones que tienen datos de precipitación son las siguientes: 102,103,106,107,108,24,39,54,56,59

In [6]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
899,28,79,107,89,28079107_89_98,2020,5,1,0.0,V,...,0.0,V,0.0,V,0.0,V,0.0,V,0.0,V


In [7]:
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
print(DataTemp.head(1))
print(DataPrec.head(1))

      PUNTO_MUESTREO   ANO  MES  DIA   H01   H02   H03   H04   H05   H06  ...  \
1147  28079109_83_98  2020    5    1  13.4  13.0  12.7  12.2  11.8  11.7  ...   

       H15   H16   H17   H18   H19   H20   H21   H22   H23   H24  
1147  20.5  21.5  22.3  23.1  22.3  20.3  18.8  17.7  17.0  16.4  

[1 rows x 28 columns]
     PUNTO_MUESTREO   ANO  MES  DIA  H01  H02  H03  H04  H05  H06  ...  H15  \
899  28079107_89_98  2020    5    1  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

     H16  H17  H18  H19  H20  H21  H22  H23  H24  
899  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 28 columns]


In [8]:
TableTemp = DataTemp


In [9]:
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TableTemp['ANO']= TableTemp['ANO'].map(str)
TableTemp['MES']= TableTemp['MES'].map(str)
TableTemp['DIA']= TableTemp['DIA'].map(str)
TableTemp.head(1)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
1147,28079109_83_98,2020,5,1,13.4,13.0,12.7,12.2,11.8,11.7,...,20.5,21.5,22.3,23.1,22.3,20.3,18.8,17.7,17.0,16.4


In [10]:
TableTemp['ds'] = TableTemp['ANO'] + '0' + TableTemp['MES'] + TableTemp['DIA']
TableTemp.head(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
1147,28079109_83_98,2020,5,1,13.4,13.0,12.7,12.2,11.8,11.7,...,21.5,22.3,23.1,22.3,20.3,18.8,17.7,17.0,16.4,2020051
1148,28079109_83_98,2020,5,2,15.8,15.5,15.1,14.6,14.0,13.8,...,26.0,26.9,26.4,26.4,25.0,23.7,22.5,21.4,20.2,2020052
1149,28079109_83_98,2020,5,3,19.4,18.8,18.3,17.7,17.1,16.5,...,29.0,29.4,30.1,29.5,28.0,27.2,26.2,24.6,23.5,2020053
1150,28079109_83_98,2020,5,4,22.7,21.8,20.6,19.8,19.2,18.6,...,28.0,30.2,30.9,29.6,27.9,25.8,23.1,21.0,20.0,2020054
1151,28079109_83_98,2020,5,5,19.9,19.0,18.2,17.0,16.4,16.0,...,23.9,24.7,24.7,24.6,22.7,21.4,19.9,19.0,17.9,2020055


# Obtenemos TableTemp1

In [11]:
TableTemp1 = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TableTemp1.head(50)

,H01
ds,
2020051,13.4
20200510,14.2
20200511,14.4
20200512,13.7
20200513,10.7
20200514,15.3
20200515,13.1
20200516,12.5
20200517,14.4


In [12]:
TableTemp1 = TableTemp1.reset_index()

In [13]:
TableTemp1['ds'] = TableTemp1['ds'] + '-' + '010000'
TableTemp1.head()

,ds,H01
0,2020051-010000,13.4
1,20200510-010000,14.2
2,20200511-010000,14.4
3,20200512-010000,13.7
4,20200513-010000,10.7


In [14]:
TableTemp1['ds'] = pd.to_datetime(TableTemp1['ds'], format='%Y%m%d-%H%M%S')

In [34]:
TableTemp1.head(50)

,Temp
ds,
2020-05-01 01:00:00,13.4
2020-05-02 01:00:00,15.8
2020-05-03 01:00:00,19.4
2020-05-04 01:00:00,22.7
2020-05-05 01:00:00,19.9
2020-05-06 01:00:00,16.9
2020-05-07 01:00:00,20.0
2020-05-08 01:00:00,19.6
2020-05-09 01:00:00,22.1


In [16]:
TableTemp1 = TableTemp1.set_index('ds')

In [17]:
TableTemp1 = TableTemp1.sort_index()

In [33]:
TableTemp1 = TableTemp1.rename(columns={'H01':'Temp'})

# Obtenemos TableTemp2

In [18]:
TableTemp2 = pd.pivot_table(TableTemp, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TableTemp2.head(50)

,H02
ds,
2020051,13.0
20200510,13.8
20200511,14.1
20200512,13.2
20200513,10.3
20200514,14.9
20200515,12.6
20200516,12.4
20200517,13.9


In [19]:
TableTemp2 = TableTemp2.reset_index()

In [20]:
TableTemp2['ds'] = TableTemp2['ds'] + '-' + '020000'
TableTemp2.head()

,ds,H02
0,2020051-020000,13.0
1,20200510-020000,13.8
2,20200511-020000,14.1
3,20200512-020000,13.2
4,20200513-020000,10.3


In [21]:
TableTemp2['ds'] = pd.to_datetime(TableTemp2['ds'], format='%Y%m%d-%H%M%S')

In [32]:
TableTemp2.head(5)

,Temp
ds,
2020-05-01 02:00:00,13.0
2020-05-02 02:00:00,15.5
2020-05-03 02:00:00,18.8
2020-05-04 02:00:00,21.8
2020-05-05 02:00:00,19.0


In [23]:
TableTemp2 = TableTemp2.set_index('ds')

In [24]:
TableTemp2 = TableTemp2.sort_index()

In [31]:
TableTemp2 = TableTemp2.rename(columns={'H02':'Temp'})


In [29]:
# TableTemp2.head(50)

In [35]:
TableTempF = pd.concat([TableTemp1, TableTemp2])
TableTempF

,Temp
ds,
2020-05-01 01:00:00,13.4
2020-05-02 01:00:00,15.8
2020-05-03 01:00:00,19.4
2020-05-04 01:00:00,22.7
2020-05-05 01:00:00,19.9
...,...
2020-05-27 02:00:00,20.5
2020-05-28 02:00:00,20.9
2020-05-29 02:00:00,21.2


In [36]:
TableTempF = TableTempF.sort_index()
TableTempF.head(50)

,Temp
ds,
2020-05-01 01:00:00,13.4
2020-05-01 02:00:00,13.0
2020-05-02 01:00:00,15.8
2020-05-02 02:00:00,15.5
2020-05-03 01:00:00,19.4
2020-05-03 02:00:00,18.8
2020-05-04 01:00:00,22.7
2020-05-04 02:00:00,21.8
2020-05-05 01:00:00,19.9


# HASTA AQUI VOY. REVISAR SI LO SIGUIENTE SE PUEDE DESCARTAR

In [ ]:
# TableTempF = TableTempF.transpose()

In [ ]:
TableTempF = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})#'H02':np.sum})

In [ ]:
TableTempF = TableTempF.reset_index()

In [ ]:
TableTempF = TableTempF.ds.replace(hour=1)

In [ ]:
TableTempF['Periodo'] = range(0,2)

In [ ]:
TableTempF

In [ ]:
TableTempF = pd.pivot_table(TableTempF, values=['ds'],columns=['H01'], index=["Periodo"], aggfunc={'H01':'first','H02':'first'})#'H02':np.sum})

In [ ]:
# Los datos horizontales desde H01 hasta H24 deben de quedar verticales. Debemod hacer un pivottable que genere un periodo de 24 horas hacia abajo. 
# La columna PUnto muestreo ya no es necesaria. 